In [3]:
import os

os.environ['HF_HOME'] = '/mnt/data/thomas/.cache' #Used to change where to save model. Uncomment this if you want to use default location
# os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
import torch

In [4]:
import fitz
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModel
from langchain_huggingface import HuggingFaceEmbeddings

from utils.Utils import *

In [5]:
def prepare_question():
    list_questions = []
    list_questions.append("Can you differentiate a cross-listed course and elective applied course in the case of a double major?")
    list_questions.append("I would like to ask for the capstone withdrawal policy. What will be the impact when I choose to drop the capstone before the Fall 2025 term starts? Will there be any penalties associated with withdrawing, such as a 'W' notation on my transcript?")
    list_questions.append("Questions related to course history, in which one course can have multiple IDs? Like Advanced Deep Learning course. How would it be handled among different cohorts?")
    # list_questions.append("I have taken Microeconomic Analysis and Macroeconomic Analysis in Fall 2023. Am I eligible to take Financial Economics in Spring 2024?")
    # list_questions.append("If I have waived the course Computer Science I, but I want to declare Computer Science major, do I have to take Computer Science I?")
    # list_questions.append("If I have taken the following courses: Mathematical Statistics, Abstract Algebra, Linear Algebra, Multivariable Calculus, Calculus, Differential Equation, Stochastic Calculus, Statistical Learning, Discrete Mathematics, Probability, am I eligible to declare a minor in Applied Mathematics? What about majoring in Applied Mathematics?")
    # list_questions.append("I have completed these 100-level courses: Introduction to Art History and Theory, Introduction to Film History and Theory. Am I eligible to start a Capstone in my major Arts and Media Studies?")
    # list_questions.append("Is it possible that I write a book as my History Capstone?")
    # list_questions.append("Given I have finished these subjects: Advanced Deep Learning, Machine Learning, Deep Learning, Database. Are they sufficient for the Advanced Courses requirement of the computer science major?")
    list_questions.append("I have a question if I take a gap year, will my financial aid be canceled? If yes, can I still re-apply for the financial aid program when I get back to school? Thank you.")
    list_questions.append("I am aware that Fulbright has to organize online classes instead of on-campus ones. Therefore, will there be any change in the tuition fee?")
    # list_questions.append("If I major in Economics, is it sufficient that I take Scholar Development to fulfill Experiential Learning requirements?")
    # list_questions.append("Is it sufficient for me to complete 4 credits in EL if I major in Engineering?")
    # list_questions.append("Can I take all four engineering foundation courses and only two intermediate engineering courses?")
    # list_questions.append("If I have completed the course Intro to AI, will CS207 - Object-Oriented Analysis and Design and CS211 – Operating Systems suffice to fill the rest two intermediate courses?")
    # list_questions.append("Does completing both Microeconomics and Macroeconomics satisfy the core theory requirement, or is an additional course needed?")
    # list_questions.append("If I want to major in Engineering, can I choose other programming courses besides Computer Science I and Computer Science II, for instance Introduction to data visualization?")
    list_questions.append("Given I took Discrete Mathematics in Spring 2022 and earned B, then the same course in Spring 2024 and earned C-, which grades will be kept?")
    list_questions.append("Can I register 300-level math courses as exploratory courses E4?")
    list_questions.append("Can I register for more than 20 credits in a semester?")
    list_questions.append("What is the maximum number of times I can retake a course to improve my grade, and how does it affect my transcript?")
    # list_questions.append("For a student doing the literature capstone, if a student includes charts or tables, do they count toward the word limit?")
    # list_questions.append("If I failed Capstone I in the Fall 2024, can I redo it in Spring 2025?")
    # list_questions.append("If I do Literature capstone in Decolonial studies, can it be approved?")
    # list_questions.append("I am a Co24 majoring in Computer Science and taking the Database course. Does it count as major courses (difference in flowchart of CS for Co24 and Co25 onwards)?")
    list_questions.append("Can Discrete Mathematics be counted for both Computer Science and Applied Mathematics majors?")
    return list_questions


In [2]:
# === Main ===
pdf_file = load_and_clean_pdf("../data/Academic-Policy_V5.1.pdf")
test_paragraph = concatenate_paragraphs(pdf_file)
chunks = chunk_paragraphs(test_paragraph)
vectorstore = build_vectorstore(chunks, persist_path="./chroma_fulbright3")

print("✅ Vectorstore built and persisted.")

llm_pipe = load_local_llm()

list_questions = prepare_question()
ask_question(llm_pipe, vectorstore, list_questions)

NameError: name 'fitz' is not defined